## Import necessary libraries

In [25]:
import pandas as pd
import pickle
import scipy
from scipy import sparse

## Read data and quick EDA

In [26]:
weather_alarms_tfidf = pd.read_csv('data/df_weather+alarms_merged.csv')

In [27]:
weather_alarms_tfidf.shape

(203420, 70)

In [28]:
weather_alarms_tfidf.head()

,Unnamed: 0,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,...,event_day_date,event_start_hour_datetimeEpoch,event_end_hour_datetimeEpoch,event_hour_level_event_time,event_hour_level_event_datetimeEpoch,isw_report_date,isw_date_tomorrow_datetime,isw_keywords,isw_main_html_v6,isw_report_text_lemm
0,0,"Черкаси, Україна",2022-02-24,1645653600,4.9,-0.1,1.9,-0.6,83.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Черкаси, Україна",2022-02-24,1645653600,4.9,-0.1,1.9,-0.6,83.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"Черкаси, Україна",2022-02-24,1645653600,4.9,-0.1,1.9,-0.6,83.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Черкаси, Україна",2022-02-24,1645653600,4.9,-0.1,1.9,-0.6,83.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"Черкаси, Україна",2022-02-24,1645653600,4.9,-0.1,1.9,-0.6,83.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
weather_alarms_tfidf.columns

Index(['Unnamed: 0', 'city_resolvedAddress', 'day_datetime',
       'day_datetimeEpoch', 'day_tempmax', 'day_tempmin', 'day_temp',
       'day_dew', 'day_humidity', 'day_precip', 'day_precipcover',
       'day_solarradiation', 'day_solarenergy', 'day_uvindex', 'day_sunrise',
       'day_sunset', 'day_moonphase', 'hour_datetime', 'hour_datetimeEpoch',
       'hour_temp', 'hour_humidity', 'hour_dew', 'hour_precip',
       'hour_precipprob', 'hour_snow', 'hour_snowdepth', 'hour_preciptype',
       'hour_windgust', 'hour_windspeed', 'hour_winddir', 'hour_pressure',
       'hour_visibility', 'hour_cloudcover', 'hour_solarradiation',
       'hour_solarenergy', 'hour_uvindex', 'hour_severerisk',
       'hour_conditions', 'city', 'region', 'center_city_ua', 'center_city_en',
       'region_alt', 'region_id', 'event_Unnamed: 0', 'event_region_title',
       'event_region_city', 'event_all_region', 'event_start', 'event_end',
       'event_clean_end', 'event_intersection_alarm_id', 'event_start_

In [30]:
weather_alarms_tfidf.shape

(203420, 70)

## TF_IDF + CV with isw_report_text_lemm preprocessed

In [31]:
tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))

#Uncommit if you want to load exist matrix
tfidf_vector = scipy.sparse.load_npz('data/matrix/tfidf_vector_train.npz')

# #Uncommit if you want to create new one
# word_count_vector = cv.transform(weather_alarms_tfidf['isw_report_text_lemm'].values.astype('U'))
# tfidf_vector = tfidf.transform(word_count_vector)

C:\Users\dimai\AppData\Local\Temp\ipykernel_10612\3783398272.py:1: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
C:\Users\dimai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\dimai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn

In [32]:
# #Save created model and matrix
# with open("models/tfidf_vector_calculated.pkl", "wb") as handle:
#    pickle.dump(tfidf_vector, handle)
# scipy.sparse.save_npz('data/matrix/tfidf_vector_train.npz', tfidf_vector)

In [33]:
tfidf_vector

<203420x7401 sparse matrix of type '<class 'numpy.float64'>'
	with 123370706 stored elements in Compressed Sparse Row format>

## Dataset Preprocesing

In [34]:
# create dependent wariable - isAlarm

weather_alarms_tfidf['isAlarm'] = weather_alarms_tfidf['event_start_hour'].notnull().astype(int)
Y = weather_alarms_tfidf['isAlarm']
weather_alarms_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203420 entries, 0 to 203419
Data columns (total 71 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   Unnamed: 0                                      203420 non-null  int64  
 1   city_resolvedAddress                            203420 non-null  object 
 2   day_datetime                                    203420 non-null  object 
 3   day_datetimeEpoch                               203420 non-null  int64  
 4   day_tempmax                                     203420 non-null  float64
 5   day_tempmin                                     203420 non-null  float64
 6   day_temp                                        203420 non-null  float64
 7   day_dew                                         203420 non-null  float64
 8   day_humidity                                    203420 non-null  float64
 9   day_precip                

In [35]:
#Clean data

weather_alarms_tfidf_features = weather_alarms_tfidf.drop(columns=['event_all_region', 'event_start',
       'event_end', 'event_clean_end', 'event_intersection_alarm_id',
       'event_start_time', 'event_end_time', 'event_day',
       'event_feature_number_of_region', 'event_within_24_hours',
       'event_feature_number_of_alarms_within_24_hours', 'event_start_hour',
       'event_end_hour', 'event_day_date', 'event_start_hour_datetimeEpoch',
       'event_end_hour_datetimeEpoch', 'event_hour_level_event_time',
       'event_hour_level_event_datetimeEpoch','region_alt','hour_conditions','center_city_en','hour_datetime','day_sunset','day_sunrise','isAlarm','hour_precip','hour_solarradiation','hour_solarenergy','hour_uvindex', 'event_region_title', 'event_region_city', 'isw_report_date', 'isw_date_tomorrow_datetime', 'isw_keywords', 'isw_main_html_v6', 'isw_report_text_lemm', 'city', 'region', 'center_city_ua', 'hour_preciptype', 'city_resolvedAddress', 'day_datetime', 'event_Unnamed: 0'],axis=1)

In [36]:
weather_alarms_tfidf_features_clear = weather_alarms_tfidf_features
weather_alarms_tfidf_features_clear.to_csv('data/weather_alarms_tfidf_features_clear.csv')

In [37]:
weather_alarms_tfidf_features_clear.isnull().sum()

Unnamed: 0            0
day_datetimeEpoch     0
day_tempmax           0
day_tempmin           0
day_temp              0
day_dew               0
day_humidity          0
day_precip            0
day_precipcover       0
day_solarradiation    0
day_solarenergy       0
day_uvindex           0
day_moonphase         0
hour_datetimeEpoch    0
hour_temp             0
hour_humidity         0
hour_dew              0
hour_precipprob       0
hour_snow             0
hour_snowdepth        0
hour_windgust         0
hour_windspeed        0
hour_winddir          0
hour_pressure         0
hour_visibility       0
hour_cloudcover       0
hour_severerisk       0
region_id             0
dtype: int64

In [38]:
# merge final dataset with tf_idf vector

weather_alarms_tfidf_csr = scipy.sparse.csr_matrix(weather_alarms_tfidf_features_clear)
weather_alarms_tfidf_features = scipy.sparse.hstack((weather_alarms_tfidf_csr, tfidf_vector), format="csr")

In [39]:
# save matrix
scipy.sparse.save_npz('data/matrix/weather_alarms_tfidf_features.npz', weather_alarms_tfidf_features)

In [40]:
weather_alarms_tfidf_features

<203420x7429 sparse matrix of type '<class 'numpy.float64'>'
	with 128262032 stored elements in Compressed Sparse Row format>

## Devide data into train and test parts, train models and evaluate them

In [41]:
#Imports

from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import shuffle

COMMAND_ID = "4"

# Split the dataset into input features (X) and target variable (y)

X = weather_alarms_tfidf_features  
y = Y  

X, y = shuffle(X, y, random_state=42)

## Discription

#### We used TimeSeriesSplit to devide our dataset for a few small and tested each model with them. We collected reports and chose model which we think the best for our needs.

## Model 1 SGDClassifier

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)
circle = 0

for train_index, test_index in tscv.split(X):

    # Access the train and test data for X and y
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print(X_train.shape)
    print(X_test.shape)

    sgd = SGDClassifier()

    sgd.fit(X_train, y_train)
    sgd_pred = sgd.predict(X_test)
    sgd_accuracy = accuracy_score(y_test, sgd_pred)
    sgd_report = classification_report(y_test, sgd_pred)

    print("SGDClassifier Accuracy:", sgd_accuracy)
    print("SGDClassifier Report:\n", sgd_report)

    # Save model
    MODEL_NAME = "sgd"
    VERSION = "1."+str(circle)
    with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
        pickle.dump(sgd, f)

    circle=circle+1

## Model 2 LogisticRegression

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)
circle = 0

for train_index, test_index in tscv.split(X):

    # Access the train and test data for X and y
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print(X_train.shape)
    print(X_test.shape)

    lr = LogisticRegression()

    lr.fit(X_train, y_train)
    lr_pred = lr.predict(X_test)
    lr_accuracy = accuracy_score(y_test, lr_pred)
    lr_report = classification_report(y_test, lr_pred)

    print("LogisticRegression Accuracy:", lr_accuracy)
    print("LogisticRegression Report:\n", lr_report)

    # Save model
    MODEL_NAME = "logreg"
    VERSION = "1."+str(circle)
    with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
        pickle.dump(lr, f)

    circle=circle+1

## Model 3 GaussianNB

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)
circle = 0

for train_index, test_index in tscv.split(X):

    # Access the train and test data for X and y
    X_train, X_test = X[train_index].toarray(), X[test_index]
    y_train, y_test = y[train_index].toarray(), y[test_index]

    print(X_train.shape)
    print(X_test.shape)

    gnb = GaussianNB()

    gnb.fit(X_train, y_train)
    gnb_pred = gnb.predict(X_test)
    gnb_accuracy = accuracy_score(y_test, gnb_pred)
    gnb_report = classification_report(y_test, gnb_pred)

    print("GaussianNB Accuracy:", gnb_accuracy)
    print("GaussianNB Report:\n",gnb_report)

    #Save model
    MODEL_NAME = "gnb"
    VERSION = "1."+str(circle)
    with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
        pickle.dump(gnb, f)

    circle=circle+1

## Model 4 RandomForestClassifier

In [ ]:
# Train model with a full dataset after we understood that it's the best model for us
# And it's best to use train_test_split here, but we already used TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=3, max_train_size=152565, test_size=50855)

for train_index, test_index in tscv.split(X):
    # Access the train and test data for X and y
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

print(X_train.shape)
print(X_test.shape)

rf = RandomForestClassifier()

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_report = classification_report(y_test, rf_pred)

print("RandomForestClassifier Accuracy:", rf_accuracy)
print("RandomForestClassifier Report:\n", rf_report)

# Save model
MODEL_NAME = "rf"
VERSION = "3.0f"
with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
    pickle.dump(rf, f)

## Model 5 SVC

In [ ]:
tscv = TimeSeriesSplit(n_splits=10)
circle = 0

for train_index, test_index in tscv.split(X):

    # Access the train and test data for X and y
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print(X_train.shape)
    print(X_test.shape)

    svm = SVC()

    svm.fit(X_train, y_train)
    svm_pred = svm.predict(X_test)
    svm_accuracy = accuracy_score(y_test, svm_pred)
    svm_report = classification_report(y_test, svm_pred)

    print("SVC Accuracy:", svm_accuracy)
    print("SVC Report:\n", svm_report)

    # Save model
    MODEL_NAME = "svm"
    VERSION = "2."+str(circle)
    with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
        pickle.dump(svm, f)

    circle=circle+1

## Model 6 KNeighborsClassifier

In [ ]:
tscv = TimeSeriesSplit(n_splits=20)
circle = 0

for train_index, test_index in tscv.split(X):
    # Access the train and test data for X and y
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print(X_train.shape)
    print(X_test.shape)

    knn = KNeighborsClassifier()

    knn.fit(X_train, y_train)
    knn_pred = knn.predict(X_test)
    knn_accuracy = accuracy_score(y_test, knn_pred)
    knn_report = classification_report(y_test, knn_pred)

    print("KNeighborsClassifier Accuracy:", knn_accuracy)
    print("KNeighborsClassifier Report:\n", knn_report)

    # Save model
    MODEL_NAME = "knn"
    VERSION = "2."+str(circle)
    with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
        pickle.dump(knn, f)

    circle=circle+1

### RandomForestClassifier Tuning

This code is work no working, because it's really heavy for our users PC. But we tried...

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

tscv = TimeSeriesSplit(n_splits=3, max_train_size=152565, test_size=50855)

# Define hyperparameters to tune
params = {
    'n_estimators': randint(100, 1000),
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'class_weight': ['balanced', None]
}

best_accuracy = 0
best_rf = None

for train_index, test_index in tscv.split(X):
    # Access the train and test data for X and y
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Perform randomized search for hyperparameter tuning
    rf = RandomForestClassifier()
    search = RandomizedSearchCV(rf, params, n_iter=10, scoring='accuracy', n_jobs=-1, cv=tscv)
    search.fit(X_train, y_train)

    # Get best model and evaluate on test set
    best_rf = search.best_estimator_
    rf_pred = best_rf.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_pred)
    if rf_accuracy > best_accuracy:
        best_accuracy = rf_accuracy
        best_rf = best_rf

    rf_report = classification_report(y_test, rf_pred)

    print("RandomForestClassifier Accuracy:", rf_accuracy)
    print("RandomForestClassifier Report:\n", rf_report)

# Save best model
MODEL_NAME = "rf"
VERSION = "3.0f"
with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
    pickle.dump(best_rf, f)